In [ ]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

from tensorflow.keras.layers import Layer

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import  keras.backend as K

class Attention(Layer):

  def __init__(self ,   return_sequences):
    super(Attention,self).__init__()
    self.return_sequences = return_sequences
     
  
  def build(self,input_shape) :  

    w_q_init  = tf.random_normal_initializer()
    w_k_init = tf.random_normal_initializer()
    w_v_init = tf.random_normal_initializer()

    embedding_size  = input_shape[-1]

    # print(embedding_size)
    self.w_q  = tf.Variable(name = 'query ' , initial_value=w_q_init(shape= (embedding_size , 36)), trainable =  True)
    self.w_k = tf.Variable(name = 'key ' , initial_value=w_k_init(shape= (embedding_size , 36)), trainable =  True )
    self.w_v = tf.Variable(name = 'value ' , initial_value=w_v_init(shape= (embedding_size , 36)), trainable =  True )

    # Shape of the subarrays formed is  (embedding_size , self.inputs)
    super(Attention , self).build(input_shape)

  def call(self , input_sequence) : 
     
    #  print(input_sequence[0])
     print(self.w_q.shape , self.w_k.shape  , input_sequence.shape)
     
     self.q = K.dot(input_sequence , self.w_q)
     self.k = K.dot(input_sequence , self.w_k)
     self.v = K.dot(input_sequence , self.w_v) 
    
    #  print(self.q.shape , self.k.shape,self.v.shape)

     dim_k = self.k.shape[-1]

     t  = tf.matmul(self.q, tf.transpose(self.k , perm = [0,2,1]))
     
     return tf.matmul(K.softmax(tf.matmul(self.q, tf.transpose(self.k , perm = [0,2,1]))/6), self.v)

def get_config(self):
        return super(Attention,self).get_config()


In [ ]:

from keras.datasets import imdb 
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding , Bidirectional , LSTM , Dropout, Dense,TimeDistributed,Flatten
import numpy as np

data = open("book.txt").read()

corpus = data.lower().split("\n")

tokenizer = Tokenizer()

tokenizer.fit_on_texts(corpus)

total_words = len(tokenizer.word_index) + 1
input_sequences=[]

for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

max_sequence_len = max((len(x) for x in input_sequences))
input_sequences = np.array(sequence.pad_sequences(input_sequences,maxlen = max_sequence_len,padding='pre'))
print(input_sequences.shape)

predictors,label = input_sequences[:,:-1],input_sequences[:,-1]

label = tf.keras.utils.to_categorical(label,num_classes=total_words)
n = len(predictors)

def train_val_test_split(predictors,label): 
  x_train = predictors[:(int)(0.7*n)]
  y_train = label[:(int)(0.7*n)]
  x_val = predictors[(int)(0.7*n):(int)(0.8*n)]
  y_val = label[(int)(0.7*n):(int)(0.8*n)]
  x_test = predictors[(int)(0.8*n):n]
  y_test = label[(int)(0.8*n):n]
  return x_train,y_train,x_val,y_val,x_test,y_test

x_train,y_train,x_val,y_val,x_test,y_test = train_val_test_split(predictors,label)
print(predictors.shape)
print(x_train.shape)
print(x_test.shape)

(20205, 17)
(20205, 16)
(14143, 16)
(4041, 16)


In [ ]:
model = tf.keras.models.Sequential()
model.add(Embedding(total_words,100, input_length = max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Attention(return_sequences=False))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(total_words/2, activation='relu'))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.summary()

(300, 36) (300, 36) (None, 16, 300)
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 16, 100)           261500    
                                                                 
 bidirectional_12 (Bidirecti  (None, 16, 300)          301200    
 onal)                                                           
                                                                 
 attention_10 (Attention)    (None, 16, 36)            32400     
                                                                 
 dropout_10 (Dropout)        (None, 16, 36)            0         
                                                                 
 flatten_2 (Flatten)         (None, 576)               0         
                                                                 
 dense_20 (Dense)            (None, 1307)              754139    
                 

In [ ]:
history3d=model.fit(predictors, label,
          batch_size=100,
          epochs=60, validation_data = [x_val,y_val],verbose=1)
# print(history3d.history['loss'])
# print(history3d.history['accuracy'])

Epoch 1/60
(300, 36) (300, 36) (None, 16, 300)
(300, 36) (300, 36) (None, 16, 300)
203/203 [==============================] - 11s 30ms/step - loss: 6.2615 - accuracy: 0.0496 - val_loss: 5.9750 - val_accuracy: 0.0559
Epoch 2/60
203/203 [==============================] - 5s 27ms/step - loss: 5.9108 - accuracy: 0.0543 - val_loss: 5.7170 - val_accuracy: 0.0609
Epoch 3/60
203/203 [==============================] - 5s 25ms/step - loss: 5.7089 - accuracy: 0.0592 - val_loss: 5.6455 - val_accuracy: 0.0693
Epoch 4/60
203/203 [==============================] - 5s 23ms/step - loss: 5.5426 - accuracy: 0.0658 - val_loss: 5.3894 - val_accuracy: 0.0732
Epoch 5/60
203/203 [==============================] - 5s 23ms/step - loss: 5.3477 - accuracy: 0.0818 - val_loss: 5.1380 - val_accuracy: 0.0940
Epoch 6/60
203/203 [==============================] - 5s 23ms/step - loss: 5.1272 - accuracy: 0.1032 - val_loss: 4.9371 - val_accuracy: 0.1237
Epoch 7/60
203/203 [==============================] - 5s 23ms/step - 

In [ ]:
seed_text = "Drops of rain"
next_words = 10
for _ in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = sequence.pad_sequences([token_list],maxlen = max_sequence_len-1,padding='pre')
  predicted = np.argmax(model.predict(token_list,verbose=0), axis=-1)
  # print(predicted)
  output_word = ""
  for word,index in tokenizer.word_index.items():
    if index == predicted:
      # print(word)
      output_word = word
      break
  seed_text+=" "+output_word

print(seed_text)

score, acc = model.evaluate(x_test, y_test,batch_size=100)
print(acc)

(300, 36) (300, 36) (None, 16, 300)
Drops of rain could be heard hitting the pane which made him feel
41/41 [==============================] - 0s 11ms/step - loss: 0.2701 - accuracy: 0.9183
0.9183370471000671
